In [ ]:
import sympy as sp
from sympy import S, collect, expand, factor, Wild
from sympy.abc import x, y
y = sp.Symbol('y')
x = sp.Symbol('x')

Polynomial representation to binary representation 

In [ ]:
def binary_conv(state):
  binary=[0 for i in range(16)]
  binary1 = [0 for i in range(128)]
  for i in range(4):
    for j in range(4):
      a = state[i][j].all_coeffs()
      binary[4*i+j] = a
      if len(a) != 8:
        for k in range(8-len(a)):
          a.insert(k,0)
      binary[4*i+j] = a
  for i in range(16):
    for j in range(8):
      binary1[8*i+j] = binary[i][j]
  return binary1



gcd function (Used for inverses)




In [ ]:
#This function finds the gcd of 2 polynomials 
def gcd(a,b):
  (throwaway,a) = sp.polys.polytools.reduced(a,[b], modulus=2)

  if a == 0:
    return (b, 0, 1)
  else:
    (throwaway,bmoda) = sp.polys.polytools.reduced(b,[a], modulus=2)
    g, y, x = gcd(bmoda,a) 
    (g,x,y) = (g, x - (b // a) * y, y)
    x = sp.Poly.from_poly(x,modulus=2)
    return (g,x,y)
#Through a recursive algorithm it returns gcd(a,b) = g as well as x and y
#such that a*x + b*y = g



Inverse function


In [ ]:
#As we have the gcd and the linear combination of a and b s.t. it equals the gcd
#The inverse function simply checks if there exists an inverse and if it does returns
#x such that a*x + b*y = 1 (which is the onverse of a mod b)
def inv(a,b):
    g, x, y = gcd(a,b)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        #Here I have a*x + b*y = 1 therefore x mod m is the inverse of a
        return x
g = sp.Poly(x**4+x**3+x**2+x+1, modulus=2)
h = sp.Poly(x**8+1, modulus=2)
inv(g,h)

Poly(x**6 + x**3 + x, x, modulus=2)

τ

In [ ]:
def tau(byte,poly):
  trans = byte+poly
  return trans

σ

In [ ]:
def sigma(byte):
  m = sp.Poly(x**8+x**4+x**3+x+1) # define m 
  if byte == 0:
    g = sp.Poly(x**6+x**5+x+1, modulus=2) # s-box when input is 0
  else:
    g = tau(inv(byte,m)*sp.Poly(x**4+x**3+x**2+x+1, modulus=2),sp.Poly(x**6+x**5+x+1, modulus=2)) 
    # s-box for all other functions according to the formula
    (a,g) = sp.polys.polytools.reduced(g,[x**8+1],modulus=2) # reduce mod x^8+1
  return(g) #returns encrypted byte

sigma(sp.Poly(x**4+x**3+1, modulus=2))

Poly(x**7 + x**6 + x**4 + x**2, x, modulus=2)

In [ ]:
def sigma_inv(byte):
  m = sp.Poly(x**8+x**4+x**3+x+1,modulus=2)
  if byte == sp.Poly(x**6+x**5+x+1, modulus=2):
    byte_dec = 0
  else: 
    byte_dec = tau(byte*sp.Poly(x**6+x**3+x,x,modulus=2),sp.Poly(x**2+1,modulus=2)) # inverse translation
    (a,byte_dec) = sp.polys.polytools.reduced(byte_dec,[x**8+1],modulus=2)
    byte_dec = inv(byte_dec,m) # multiplication by inverse of x^4+x^3+x^2+x+1
  return byte_dec

In [ ]:
sigma_inv(sigma(sp.Poly(x**4+x**3+1, modulus=2))) #Testing sigma inverse works

Poly(x**4 + x**3 + 1, x, modulus=2)

ξ

In [ ]:
def xi(word):
  g = [0,0,0,0] #initialize output array
  for i in range(4):
    g[i] = sigma(word[(i+1) % 4]) #we permute each byte in the word and map it through sigma

  return(g)

word = [sp.Poly(x**4+x, modulus=2),sp.Poly(x**5+x**2, modulus=2),sp.Poly(x**6+x**3, modulus=2),sp.Poly(x**7+1, modulus=2)] #test word
xi(word) 

[Poly(x**5 + x**4 + x**2 + x, x, modulus=2),
 Poly(x**6 + x**4 + x, x, modulus=2),
 Poly(x**3 + x**2, x, modulus=2),
 Poly(x**7 + x**6 + x**3 + 1, x, modulus=2)]



μ,ν

In [ ]:
def mu(word):
  c=[sp.Poly(x,x,modulus=2),sp.Poly(1,x,modulus=2),sp.Poly(1,x,modulus=2),sp.Poly(x+1,x,modulus=2)] #here we define the polynomial c from the lecture notes in F2[x]
  g=0
  
  for i in range(4): #we get our word represented as a single polynoial (g) in (F2[x]/(m) [y]/(y^4 + 1))
    w=word[i]*(sp.Poly(y**i,y,modulus=2))
    g=g+w
    g=sp.Poly.from_poly(g,modulus=2)
  
  q=0 
  #for mu
  for i in range(4): #we get c represented as a single polynoial in (F2[x]/(m) [y]/(y^4 + 1)) which is used to get μ(word)
    w=c[i]*(sp.Poly(y**i,y,modulus=2))
    q=q+w
    q=sp.Poly.from_poly(q,modulus=2) #here q is the (F2[x]/(m) [y]/(y^4 + 1)) representation of c

  word_enc_mu=q*(g) #μ(word) is defined as g*c
  word_enc_mu=sp.polys.polytools.reduced(word_enc_mu,[y**4+1],modulus=2) 
  word_enc_mu=sp.polys.polytools.reduced(word_enc_mu[1],[x**8+x**4+x**3+x+1],modulus=2) #we get μ(word) mod (y^4+1)(m)

  word_enc_mu1=word_enc_mu[1]

  #Up to this point μ(word) is still represented in (F2[x]/(m) [y]/(y^4 + 1))
  #We want to put it back in an array format of 4 bytes each in F2[x]/(m)

  word_enc_mu1=sp.collect(word_enc_mu1.as_expr(),y) #First we collect all the like terms for each power of y


  word_enc_mu1=sp.Poly(word_enc_mu1,y).all_coeffs() #Next we take the coefficients of each y^i and put it in an array format



  if len(word_enc_mu1) != 4:
    for i in range(4-len(word_enc_mu1)):
      word_enc_mu1.insert(0,0)


  for i in range(4): #The polynomials are in expr format while for our purposes we wanr Poly format so we reformat it 
    word_enc_mu1[i] = sp.Poly(word_enc_mu1[i],x,modulus=2)


  word_enc_mu1.reverse()


  return(word_enc_mu1) #we return μ(word)
  
#test using the word defined above
print(mu(word))
a = mu(mu(mu(mu(word))))
print(a)

[Poly(x**7 + 1, x, modulus=2), Poly(x + 1, x, modulus=2), Poly(x**5 + x**4 + x**3 + x**2 + x, x, modulus=2), Poly(x**6 + x + 1, x, modulus=2)]
[Poly(x**4 + x, x, modulus=2), Poly(x**5 + x**2, x, modulus=2), Poly(x**6 + x**3, x, modulus=2), Poly(x**7 + 1, x, modulus=2)]


In [ ]:
def nu(word):
  return(mu(mu(mu(word)))) #as nu = mu^3

ρ

In [ ]:
def rho(state):
  state_enc = [[state[(j-i)][j] for j in range(4)] for i in range(4)] 
  #switch around the bytes in words
  state_enc[1], state_enc[3] = state_enc[3], state_enc[1] 
  #swap two encrypted words with each other due to switching around
  return(state_enc)

Key state generator (sequence of 43 wn's)

In [ ]:
def state_gen(key):
  wn = [0 for i in range(44)] #initiate sequence wn as a list
  wn[0] = key[0] #substitute first 4 entries by the key entries
  wn[1] = key[1]
  wn[2] = key[2]
  wn[3] = key[3]

  for i in range(40): #use the formula to get the rest of the terms (between 4 and 43)
    p=[0,0,0,0] #initialise a word p
    if i%4 == 0: #procedure to get wn[i] for i divisible by 4
      X = [sp.Poly(x**int((i/4)),x,modulus=2),0,0,0]
      k = xi(wn[i+3])
      w = wn[i]
      for j in range(4): #this loop does vector addition
        p[j] = X[j] + k[j] + w[j] 
        pj = sp.polys.polytools.reduced(p[j],[x**8+x**4+x**3+x+1],modulus=2) # each byte in word p taken mod m
        p[j] = pj[1] #2nd entry of pj is the remainder mod m
      wn[i+4] = p #plug into the sequence wn
    else:
      w1 = wn[i+3]
      w2 = wn[i]
      for j in range(4): #this loop does vector addition
        p[j] = w1[j] + w2[j] # procedureto get wn[i] for i not divisible by 4
        pj = sp.polys.polytools.reduced(p[j],[x**8+x**4+x**3+x+1],modulus=2) # each byte in word p taken mod m
        p[j] = pj[1] #2nd entry of pj is the remainder mod m
      wn[i+4] = p #plug into the sequence

  return wn


#example words for the key
a = [sp.Poly(x**2+x+1,modulus=2), sp.Poly(x**2+x+1,modulus=2), sp.Poly(x**2+x+1,modulus=2),sp.Poly(x**2+x+1,modulus=2)]
b = [sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2)]
c = [sp.Poly(x**5+x**4+x**2+1, modulus=2), sp.Poly(x**5+x**4+x**2+1, modulus=2),sp.Poly(x**5+x**4+x**2+1, modulus=2),sp.Poly(x**5+x**4+x**2+1, modulus=2)]
d = [sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2)]
key1 = [a,b,c,d] #creating key from words
key2 = [d,c,b,a]
b = state_gen(key1) #test run
print(b)

[[Poly(x**2 + x + 1, x, modulus=2), Poly(x**2 + x + 1, x, modulus=2), Poly(x**2 + x + 1, x, modulus=2), Poly(x**2 + x + 1, x, modulus=2)], [Poly(x**4 + x**3 + 1, x, modulus=2), Poly(x**4 + x**3 + 1, x, modulus=2), Poly(x**4 + x**3 + 1, x, modulus=2), Poly(x**4 + x**3 + 1, x, modulus=2)], [Poly(x**5 + x**4 + x**2 + 1, x, modulus=2), Poly(x**5 + x**4 + x**2 + 1, x, modulus=2), Poly(x**5 + x**4 + x**2 + 1, x, modulus=2), Poly(x**5 + x**4 + x**2 + 1, x, modulus=2)], [Poly(x**7 + 1, x, modulus=2), Poly(x**7 + 1, x, modulus=2), Poly(x**7 + 1, x, modulus=2), Poly(x**7 + 1, x, modulus=2)], [Poly(x**3 + x, x, modulus=2), Poly(x**3 + x + 1, x, modulus=2), Poly(x**3 + x + 1, x, modulus=2), Poly(x**3 + x + 1, x, modulus=2)], [Poly(x**4 + x + 1, x, modulus=2), Poly(x**4 + x, x, modulus=2), Poly(x**4 + x, x, modulus=2), Poly(x**4 + x, x, modulus=2)], [Poly(x**5 + x**2 + x, x, modulus=2), Poly(x**5 + x**2 + x + 1, x, modulus=2), Poly(x**5 + x**2 + x + 1, x, modulus=2), Poly(x**5 + x**2 + x + 1, x, mo

AES step for encryption

In [ ]:
def AES_step(state,key_n):
  state_enc_n = [[tau(state[i][j],key_n[i][j]) for j in range(4)] for i in range(4)] 
  # translation by key_n

  for i in range(4):
    for j in range(4):
      state_enc_n[i][j] = sigma(state_enc_n[i][j]) # image under sigma

  state_enc_n = rho(state_enc_n) # image under rho

  for i in range(4):
    state_enc_n[i] = mu(state_enc_n[i]) # image under mu

  return(state_enc_n)

Encryption


In [ ]:
def AES_encrypt(state,key): #the encryption function
  key_aes = state_gen(key) # generate full sequence (wj) from the key
  state_enc_aes = state
  key_aes_n = [[key_aes[4*i],key_aes[4*i+1],key_aes[4*i+2],key_aes[4*i+3]] for i in range(11)]

  for i in range(9):
    state_enc_aes = AES_step(state_enc_aes,key_aes_n[i])
  
  
  state_enc_aes = [[tau(state_enc_aes[i][j],key_aes_n[9][i][j]) for j in range(4)] for i in range(4)] 
  # translation by key_n
  
  for i in range(4):
    for j in range(4):
      state_enc_aes[i][j] = sigma(state_enc_aes[i][j]) # image under sigma

  state_enc_aes = rho(state_enc_aes) # image under rho

  state_enc_aes = [[tau(state_enc_aes[i][j],key_aes_n[10][i][j]) for j in range(4)] for i in range(4)] 
  # translation by key_n

  return(state_enc_aes)

  

k = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
msg = [[sp.Poly(x**7,modulus=2),0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
print(AES_encrypt(msg,k))
msg_encrypted  = AES_encrypt(msg,k)

a = binary_conv(msg_encrypted)
print(a)


[[Poly(x**5 + x**4 + x**3 + x, x, modulus=2), Poly(x**7 + x**6 + x**4 + x**2 + x + 1, x, modulus=2), Poly(x**7 + x**3 + x**2 + x, x, modulus=2), Poly(x**6 + x**5 + x**4 + x, x, modulus=2)], [Poly(x**6 + x**5 + x**3 + x**2, x, modulus=2), Poly(x**4 + x**3 + x**2 + x, x, modulus=2), Poly(x**7 + x**6, x, modulus=2), Poly(x**5 + x**3 + x + 1, x, modulus=2)], [Poly(x**6 + x**5 + x**4 + x**3 + x**2 + x, x, modulus=2), Poly(x**7 + x**5 + x**4 + x**3 + x**2 + x + 1, x, modulus=2), Poly(x**7 + x**6 + x**5 + x**3 + 1, x, modulus=2), Poly(x**5 + x**3 + x + 1, x, modulus=2)], [Poly(x**5 + x + 1, x, modulus=2), Poly(x**7 + x**6 + x**4 + x**3 + 1, x, modulus=2), Poly(x**7 + x**6 + x**5 + x**3 + x**2, x, modulus=2), Poly(x**5 + x**4 + x**2, x, modulus=2)]]
[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1

AES step for decryption

In [ ]:
def AES_step_dec(state,key_n):
  state_dec_n = [[tau(state[i][j],key_n[i][j]) for j in range(4)] for i in range(4)] # translation by key_n
  
  for i in range(4):
      state_dec_n[i] = nu(state_dec_n[i])

  state_dec_n = rho(rho(rho(state_dec_n))) # image under rho^3 = rho^-1


  for i in range(4):
    for j in range(4):
      state_dec_n[i][j] = sigma_inv(state_dec_n[i][j]) # image under sigma^-1

  return(state_dec_n)

Decryption

In [ ]:
def AES_decrypt(state,key):
  key_aes = state_gen(key) # generate full sequence (wj) from the key
  state_dec_aes = state
  key_aes_n = [[key_aes[4*i],key_aes[4*i+1],key_aes[4*i+2],key_aes[4*i+3]] for i in range(11)]
  
  state_dec_aes = [[tau(state[i][j],key_aes_n[10][i][j]) for j in range(4)] for i in range(4)]

  state_dec_aes = rho(rho(rho(state_dec_aes)))

  for i in range(4):
    for j in range(4):
      state_dec_aes[i][j] = sigma_inv(state_dec_aes[i][j])


  for j in range(9):
    i = 9-j 
    state_dec_aes = AES_step_dec(state_dec_aes,key_aes_n[i])

  state_dec_aes = [[tau(state_dec_aes[i][j],key_aes_n[0][i][j]) for j in range(4)] for i in range(4)]


  

  return(state_dec_aes)

print(msg)
print(AES_decrypt(msg_encrypted,k)) #decrypting the encrypted message from earlier

[[Poly(x**7, x, modulus=2), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[Poly(x**7, x, domain='ZZ'), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


In [ ]:
# as a state:
Hello_World = [[sp.Poly(x**6+x**3,x,modulus=2),sp.Poly(x**6+x**5+x**2+1,x,modulus=2),sp.Poly(x**6+x**5+x**3+x**2,x,modulus=2),sp.Poly(x**6+x**5+x**3+x**2,x,modulus=2)],[sp.Poly(x**6+x**5+x**3+x**2+x+1,x,modulus=2),sp.Poly(x**5,x,modulus=2),sp.Poly(x**6+x**4+x**2+x+1,x,modulus=2),sp.Poly(x**6+x**5+x**3+x**2+x+1,x,modulus=2)],[sp.Poly(x**6+x**5+x**4+x,x,modulus=2),sp.Poly(x**6+x**5+x**3+x**2,x,modulus=2),sp.Poly(x**6+x**5+x**2,x,modulus=2),sp.Poly(x**5,x,modulus=2),],[sp.Poly(x**5,x,modulus=2),sp.Poly(x**5,x,modulus=2),sp.Poly(x**5,x,modulus=2),sp.Poly(x**5,x,modulus=2)]]
a = [sp.Poly(x**2+x+1,modulus=2), sp.Poly(x**2+x+1,modulus=2), sp.Poly(x**2+x+1,modulus=2),sp.Poly(x**2+x+1,modulus=2)]
b = [sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2), sp.Poly(x**4+x**3+1, modulus=2)]
c = [sp.Poly(x**5+x**4+x**2+1, modulus=2), sp.Poly(x**5+x**4+x**2+1, modulus=2),sp.Poly(x**5+x**4+x**2+1, modulus=2),sp.Poly(x**5+x**4+x**2+1, modulus=2)]
d = [sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2),sp.Poly(x**7+1, modulus=2)]
key = [a,b,c,d] #creating key from words
Word_enc = AES_encrypt(Hello_World,key)
print(binary_conv(Word_enc))
Word = AES_decrypt(Word_enc,key)
print(binary_conv(Word))


[0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
